In [ ]:
from __future__ import print_function,division
import sys
sys.path.append('/home/oem/vision')
sys.path.append('/home/oem/miniconda3/lib/python3.9/site-packages/skimage')


import numpy as np
import random
import os
from os.path import  join
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch.optim.lr_scheduler import ExponentialLR as ExponentialLR

import torchvision
import matplotlib.pyplot as plt

import medpy
import PIL
import json

from TMSNet_4Decoders_Train_Test import train, test
from TMSNet_4Decoders import TMSNet_4Decoders
import pdb

from DataLoader import get_data


random.seed(0)
torch.manual_seed(0)

device=torch.device("cuda:1")  

class option:
    def __init__(self,PathToSaveTrainedModels,Dataset,device=torch.device("cuda:0")):
        
        
        self.Dataset=Dataset       
        if self.Dataset=='Atrium2018':
            self.InputSize = 128           
            self.weight_decay=0.001 
            self.batchSize=8
        elif self.Dataset=='Atrium2013':
            self.InputSize = 192
            self.weight_decay=0.001 
            self.batchSize=8 
        elif self.Dataset=='PrivateDataset':
            self.InputSize = 192
            self.weight_decay=0.001 
            self.batchSize=8             
        elif self.Dataset=='DRIVE':
            self.InputSize = 192*3
            self.weight_decay=0.001 
            self.batchSize=1  
        elif self.Dataset=='IOSTAR':
            self.InputSize = 192*3
            self.weight_decay=0.001 
            self.batchSize=1    
        elif self.Dataset=='STARE':
            self.InputSize = 192*3
            self.weight_decay=0.001 
            self.batchSize=1             

            
        self.lr =0.0005 
        self.cuda = True
        self.nEpochs = 100
        self.gammaForScheduler=0.95 
        self.FolderName='8'
        self.threads=0
        self.testBatchSize=1
        self.PathToSaveTrainedModels=PathToSaveTrainedModels
        self.device=device



        
class TrainingParameters:
    def __init__(self,device,training_data_loader_All,training_data_loader_1,training_data_loader_2,training_data_loader_3,training_data_loader_4,val_data_loader_All,val_data_loader_1,val_data_loader_2,val_data_loader_3,val_data_loader_4,testing_data_loader):
        
        self.initial_seed=23789 
        self.training_data_loader_All=training_data_loader_All
        self.training_data_loader_1=training_data_loader_1
        self.training_data_loader_2=training_data_loader_2
        self.training_data_loader_3=training_data_loader_3
        self.training_data_loader_4=training_data_loader_4
             
        self.val_data_loader_All=val_data_loader_All           
        self.val_data_loader_1=val_data_loader_1
        self.val_data_loader_2=val_data_loader_2
        self.val_data_loader_3=val_data_loader_3
        self.val_data_loader_4=val_data_loader_4
        self.testing_data_loader=testing_data_loader
        self.device=device
        

        





In [ ]:
def checkpoint(epoch):
    model_out_path = join(opt.PathToSaveTrainedModels,"epoch_{}.pth".format(epoch))
    torch.save(model.state_dict(), model_out_path)
    
    
    fig, [ax1,ax2] = plt.subplots( nrows=1, ncols=2 )  # create figure & 1 axis
    ax1.plot(trainingLoss)
    ax1.set(xlabel='epochs', ylabel='TrainingLoss')  
    ax2.plot(validationLoss)
    ax2.set(xlabel='epochs', ylabel='ValidationLoss')  

    plt.savefig(join(opt.PathToSaveTrainedModels,"ErrorPlot_{}_Epoch.png".format(epoch)))
    plt.show()
    plt.close(fig)     
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def TestForNoisyInput(attacks,opt,TranPara,model,device,method,noise,TestDataset):
    for AttackType in attacks:#['Rician']: #,'Gaussian'
        opt.AttackType=AttackType

        if opt.AttackType=='Rician':
            noiseRange= [15,25,10,5,20]
        elif opt.AttackType=='Gaussian':
            noiseRange=[1,5,25,10,15,20]

        else:
            noiseRange=[0.01,0.02,0.03,0.04]

        for epsilon in noiseRange:

            test(TranPara,opt,model, device,epsilon=epsilon,method=method,noise=noise,TestDataset=TestDataset)


In [ ]:
if __name__ == '__main__':



    model=TMSNet_4Decoders(W=6).to(device) # W=8 for STACOM 2013, W=6 for DRIVE and the private dataset
    
    ## update paths accordingly for your computer
    PathToSaveTrainedModels= ---- PathToSaveTrainedModels-------
    
    
    
    Dataset='Atrium2013'
    #MainFolder='2013' # a subfolder of Dataset 
    
    
    opt=option(PathToSaveTrainedModels,Dataset,device)
    optimizer  =optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=opt.lr) 
    scheduler=ExponentialLR(optimizer,gamma=opt.gammaForScheduler) 
    
    
    dataPath= ---PathToDataFolder----  ##"/home/oem/Desktop/TMS-Net/ekler/Datasets/Atrium2013/"
    
    
    train_set=get_data(opt,dataPath,DataloaderType='training',Module='Encoder')
    training_data_loader_All = DataLoader(dataset=train_set, num_workers=0, batch_size=opt.batchSize, shuffle=True,pin_memory=True,drop_last=False)

    val_set = get_data(opt,dataPath,DataloaderType='validation')
    val_data_loader_All = DataLoader(dataset=val_set, num_workers=0, batch_size=opt.batchSize, shuffle=True,pin_memory=False)

    
    
    
    train_set = get_data(opt,dataPath,DataloaderType='training',Module='')
    training_data_loader_1 = DataLoader(dataset=train_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=True,pin_memory=False,drop_last=False)

    val_set = get_data(opt,dataPath,DataloaderType='validation')
    val_data_loader_1 = DataLoader(dataset=val_set, num_workers=0, batch_size=opt.batchSize, shuffle=True,pin_memory=False)



    
    test_set =get_data(opt,dataPath,DataloaderType='test') 
    testing_data_loader= DataLoader(dataset=test_set, num_workers=0, batch_size=opt.testBatchSize, shuffle=False,pin_memory=False)

   
    TranPara=TrainingParameters(device,training_data_loader_All,training_data_loader_1,training_data_loader_1,training_data_loader_1,training_data_loader_1,val_data_loader_All,val_data_loader_1,val_data_loader_1,val_data_loader_1,val_data_loader_1,testing_data_loader)
 


In [ ]:

it=0

trainingLoss=[]
validationLoss=[]

epsilon=0
## 30 epochs for STACOM 2013
#model.load_state_dict(torch.load(os.path.join(PathToSaveTrainedModels,'epoch_6.pth')), strict= False)


TestDataset=Dataset


for epoch in range(opt.nEpochs):


    model,optimizer, trainLoss, valLoss= train(TranPara,opt,model,epoch,optimizer)
    trainingLoss.append(trainLoss)
    validationLoss.append(valLoss)
    scheduler.step()


    it=it+1
    print("TrainingEpoch:"+str(it))
    if epoch%1 == 0:
        checkpoint(epoch)
    if epoch%1 == 0 and epoch>500:

        test(TranPara,opt,model,device,TestDataset)



In [ ]:
for method in methods:
    from QualityEstimation import QualityEstimationCorrected , DoAll
    Dataset='Atrium2013'
    Attacks=['Rician']
    print(method)

    mainPathTraining='....... /onTrainingData/'+method # path to Json files saved for training images
    mainPathTest='......../onTestData/'+method # path to Json files saved for test images
    mainPath=mainPathTest
    DoAll(Dataset,mainPath,mainPathTraining,mainPathTest,Attacks,ignoreZero=True) #True


In [ ]:
## example code to generate noisy input images for Atrium2013 datasets for a model trained with Atrium2013 dataset.
attacks=['Rician'] 
TestDataset='Atrium2013'
noise=True
TranPara.testing_data_loader= ....... # change it according to test set path
TestForNoisyInput(attacks,opt,TranPara,model,device,method,noise,TestDataset):

